# 금통위 의사록 데이터 전처리

### 금통위 의사록 섹션 분리 

KoNLPY 설치

In [ ]:
!sudo apt-get install g++ openjdk-7-jdk

In [ ]:
!sudo apt-get install python-dev; pip install konlpy

In [ ]:
 !sudo apt-get install python3-dev; pip3 install konlpy   

mecab 설치

In [ ]:
!sudo apt-get install curl

In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


eKoNLPY 설치

In [ ]:
!git clone https://github.com/entelecheia/eKoNLPy.git

In [ ]:
cd eKoNLPy

In [ ]:
!pip install .

In [ ]:
!pip install . --upgrade

금통위 의사록 섹션 분리 -> csv 파일로 저장

In [ ]:
pip install html2text

In [ ]:
import html2text
import re
from datetime import datetime, timedelta

In [ ]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [ ]:
import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

### 금통위 의사록 n-gram 구축

In [ ]:
import pandas as pd
from ekonlpy.sentiment import MPCK

def text2ngram(text) :
    mpck = MPCK()
    tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(tokens)
                
    return tokens+ ngrams

In [ ]:
import csv
import pandas as pd
from os import listdir
from os.path import isfile, join
import re

    
def preprocessing(source_folder="./text/", output_file="bok_minutes_final.csv") :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    txt_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    txt_files.sort()
    df = pd.DataFrame(columns=['date', 'minutes'])
    df["Economic Situation"] = ""
    df["Foreign Currency"] = ""
    df["Financial Markets"] = ""
    df["Monetary Policy"] = ""
    df["Participant Views"] = ""
    df["Government View"] = ""

    df["Economic Situation count"] = ""
    df["Foreign Currency count"] = ""
    df["Financial Markets count"] = ""
    df["Monetary Policy count"] = ""
    df["Participant Views count"] = ""
    df["Government View count"] = ""

    df["ngram"] = ""
    

    for txt_file in txt_files :
        with open(source_folder + txt_file, 'r') as f :
            txt = f.read()
            sections, section_texts = preprocess_minutes(txt)
            
            print(txt_file)
            
            ngrams = []
            for section_text in section_texts[1:3] :
                for sentence in section_text :
                    ngrams.append(','.join(text2ngram(sentence)))
                
            #print('@@@'.join(ngrams))
            #break

            # 파일명에서 date 추출
            date = re.search('[0-9]{4}.[0-9]{1,2}.[0-9]{1,2}', txt_file).group().replace('.','-')

            df.loc[len(df)] = [date,
                               txt,
                               '@@@'.join(section_texts[0]),
                               '@@@'.join(section_texts[1]),
                               '@@@'.join(section_texts[2]),
                               '@@@'.join(section_texts[3]),
                               '@@@'.join(section_texts[4]),
                               '@@@'.join(section_texts[5]),
                               len(section_texts[0]),
                               len(section_texts[1]),
                               len(section_texts[2]),
                               len(section_texts[3]),
                               len(section_texts[4]),
                               len(section_texts[5]),
                               '@@@'.join(ngrams)]
    
    df.to_csv(output_file, index=False)
        
    print(df.shape)
preprocessing()

금융통화위원회 의사록(2005년도 제12차)(2005.6.9).txt
2005-6-9
금융통화위원회 의사록(2005년도 제13차)(2005.6.23).txt
2005-6-23
금융통화위원회 의사록(2005년도 제14차)(2005.7.7).txt
2005-7-7
금융통화위원회 의사록(2005년도 제17차)(2005.8.11).txt
2005-8-11
금융통화위원회 의사록(2005년도 제19차)(2005.9.8).txt
2005-9-8
금융통화위원회 의사록(2005년도 제20차)(2005.9.22).txt
2005-9-22
금융통화위원회 의사록(2005년도 제21차)(2005.10.11).txt
2005-10-11
금융통화위원회 의사록(2005년도 제24차)(2005.11.10).txt
2005-11-10
금융통화위원회 의사록(2005년도 제26차)(2005.12.8).txt
2005-12-8
금융통화위원회 의사록(2005년도 제27차)(2005.12.22).txt
2005-12-22
금융통화위원회 의사록(2006년도 제10차)(2006.5.11).txt
2006-5-11
금융통화위원회 의사록(2006년도 제11차)(2006.5.25).txt
2006-5-25
금융통화위원회 의사록(2006년도 제12차)(2006.6.8).txt
2006-6-8
금융통화위원회 의사록(2006년도 제13차)(2006.6.22).txt
2006-6-22
금융통화위원회 의ᄉ

In [36]:
import pandas as pd
df = pd.read_csv("./bok_minutes_final.csv")
new_minutes_df = df[df['ngram'].notna() ]
new_minutes_df

,date,minutes,Economic Situation,Foreign Currency,Financial Markets,Monetary Policy,Participant Views,Government View,Economic Situation count,Foreign Currency count,Financial Markets count,Monetary Policy count,Participant Views count,Government View count,ngram
0,2005-6-9,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 소비재판매액 증가세가 다소 둔화되고 있으나 GDP 상의 소비와...,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,"일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 접한 연관성을 지니는지,...",NaN,NaN,NaN,19,4,160,0,0,0,"경제/NNG,경제/NNG,비해/VV,더/MAG,낫/VV,특별/NNG,요인/NNG,없..."
2,2005-7-7,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 이란의 새 대통령 취임.@@@이후 자국경제 보호주의 강화 등으로 ...,일부 위원은 6월 들어 국제유가가 크게 오르고 원화와 엔화간 동조화 현상이 뚜렷...,일부 위원은 단기적으로 대출이 급격히 증가 한 후에는 반드시 조정이 이어지고 민간...,NaN,NaN,NaN,13,5,160,0,0,0,"들/VV,국제/NNG,유가/NNG,크/VA,오르/VV,원화/NNG,엔/NNG,화간/..."
3,2005-8-11,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"일부 위원은 소비, 수출, 건설투자가 견조한 증가세를 보이고 물가도 안정세를 이...",일부 위원은 위안화 절상 이후 아시아 통화중 우리나라 원화의 절상폭이 가장 컸다...,일부 위원은 최근 장기시장금리 상승에도 불 구하고 기업 자금조달은 오히려 확대되고...,NaN,NaN,NaN,11,12,159,0,0,0,"위안/NNG,절상/NNG,원화/NNG,절상/NNG,가장/MAG,크/VA,추가/NNG..."
4,2005-9-8,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 7월 전망시보다 높은 수준을 지속하고 있는 국제유가가 내년도 경제성...,일부 위원은 최근 인도네시아 금융불안 사태가 발생한 가운데 국제 환투기세력이 인...,일부 위원은 시장이 제대로 작동한다면 비우 - 5 - 량등급의 회사채 가산금...,NaN,NaN,NaN,6,4,159,0,0,0,"금융/NNG,불안사태/NNG,발생/NNG,하/VV,가운데/NNG,국제/NNG,환투기..."
6,2005-10-11,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"일부 위원은 지방자치단체의 투자유치활동 이 활발해지고 항공기, 철도, 자동차 등의...",일부 위원은 미국과 우리나라의 정책금 리 격차가 확대되고 있는 가운데 금년 두 차...,일부 위원은 8월말 이후 우리나라 주가지수가 큰 폭 상승한 것은 경제성장에 대한...,NaN,NaN,NaN,11,4,158,0,0,0,"금리갭/NNG,확대/NNG,되/XSV,가운데/NNG,남/VV,fed/NNG,금리/N..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,2017-10-19,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 이번 경제전망을 보면 그동안 경기회복에도 불구하고 낮아졌던 근원물가상...,일부 위원은 최근 미국에서는 임금상승세가 확대되고 세제개혁에 대한 기대 감이 커지...,NaN,일부 위원은 유휴생산능력과 물가 간의 관계를 분석함에 있어서는 GDP갭뿐 만 아니...,일부 위원은 이번 회의에서는 기준금리를 현 1.25%에서 1.5%로 조정하는 것 ...,NaN,54,44,0,20,156,0,"임금/NNG,상승/NNG,확대/NNG,되/XSV,세제/NNG,개혁/NNG,기대/NN..."
253,2017-1-13,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 대내외 여건이 앞으로의 성장경로를 논하기가 어려울 만큼 매우 불확실한...,일부 위원은 미국의 시간당평균임금 상승률이 확대되고 있어 물가상승 압력 이 높아질...,"일부 위원은 관련부서에서는 금년중 가계대출이 은행, 비은행 모두 증가세가 둔화되겠...",일부 위원은 부동산경기나 금융사이클이 실물경기와 괴리되어 있는 것은 아 닌지 심도...,일부 위원은 다음 통화정책방향 결정 시까지 기준금리를 현 1.25%에서 유지 하여...,NaN,55,14,15,16,157,0,"임금/NNG,상승률/NNG,확대/NNG,되/XSV,물가/NNG,상승/NNG,압력/N..."
255,2017-11-30,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 9∼10월 실물경제 지표를 묶어서 보면 서비스업 생산이 늘어나 는 가...,일부 위원은 최근 미국의 금융·경제상황을 보면 금년 12월 미 연준이 금리를 인상...,NaN,일부 위원은 수출 증가세가 높더라도 가계부채 등 구조적 요인으로 인해 민 간소비로...,일부 위원은 지난 금통위에서 언급하였던 금리인상의 요인들이 경제흐름에 의해 확인되...,NaN,61,34,0,18,147,0,"금융경제/NNG,보/VV,fed/NNG,금리/NNG,인상/NNG,하/VV,예견/NN..."
258,2017-2-23,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 금년 1월 소비자물가가 전년동월대비 2.0% 상승하는 등 예상보 다 ...,일부 위원은 Yellen 미 연준 의장이 미국의 금리인상 지연은 바람직하지 않다 ...,NaN,일부 위원은 일반인 기대인플레이션이 물가목표보다 높은 수준을 지속해 오 다가 최근...,일부 위원은 다음 통화정책방향 결정시까지 기준금리를 현 수준 1.25%로 유 지하...,NaN,46,51,0,17,157,0,"금리/NNG,인상/NNG,지연/NNG,바람직/NNG,않/VX,언급/NNG,하/VV,..."


In [9]:
import pandas as pd
bok_minutes_csv = pd.read_csv("./bok_minutes_final.csv")
call_rate_csv = pd.read_csv("mk_call_rate_label.csv")
call_rate_csv

,date,call_rate,call_label
0,2004-12-01,3.26,0
1,2004-12-02,3.26,0
2,2004-12-03,3.26,0
3,2004-12-04,3.26,0
4,2004-12-05,3.26,0
...,...,...,...
4774,2017-12-27,1.51,increase
4775,2017-12-28,1.55,increase
4776,2017-12-29,1.58,increase
4777,2017-12-30,1.58,increase


### call_rate_csv의 'date' 형식에 맞추기(예: 2017-1-1 -> 2017-01-01)

In [23]:
def dateAddZero(date):
  year, month, day = date.split('-')
  if len(month) == 1:
    month = '0' + month
  if len(day) == 1:
    day = '0' + day
  new_date = year + '-' + month + '-' + day
  return new_date

In [ ]:
new_dates = []
for i in new_minutes_df['date']:
  new_dates.append(dateAddZero(i))
new_dates

In [46]:
new_minutes_df['date'] = new_dates
new_minutes_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date,minutes,Economic Situation,Foreign Currency,Financial Markets,Monetary Policy,Participant Views,Government View,Economic Situation count,Foreign Currency count,Financial Markets count,Monetary Policy count,Participant Views count,Government View count,ngram
0,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 소비재판매액 증가세가 다소 둔화되고 있으나 GDP 상의 소비와...,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,"일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 접한 연관성을 지니는지,...",NaN,NaN,NaN,19,4,160,0,0,0,"경제/NNG,경제/NNG,비해/VV,더/MAG,낫/VV,특별/NNG,요인/NNG,없..."
2,2005-07-07,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 이란의 새 대통령 취임.@@@이후 자국경제 보호주의 강화 등으로 ...,일부 위원은 6월 들어 국제유가가 크게 오르고 원화와 엔화간 동조화 현상이 뚜렷...,일부 위원은 단기적으로 대출이 급격히 증가 한 후에는 반드시 조정이 이어지고 민간...,NaN,NaN,NaN,13,5,160,0,0,0,"들/VV,국제/NNG,유가/NNG,크/VA,오르/VV,원화/NNG,엔/NNG,화간/..."
3,2005-08-11,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"일부 위원은 소비, 수출, 건설투자가 견조한 증가세를 보이고 물가도 안정세를 이...",일부 위원은 위안화 절상 이후 아시아 통화중 우리나라 원화의 절상폭이 가장 컸다...,일부 위원은 최근 장기시장금리 상승에도 불 구하고 기업 자금조달은 오히려 확대되고...,NaN,NaN,NaN,11,12,159,0,0,0,"위안/NNG,절상/NNG,원화/NNG,절상/NNG,가장/MAG,크/VA,추가/NNG..."
4,2005-09-08,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 7월 전망시보다 높은 수준을 지속하고 있는 국제유가가 내년도 경제성...,일부 위원은 최근 인도네시아 금융불안 사태가 발생한 가운데 국제 환투기세력이 인...,일부 위원은 시장이 제대로 작동한다면 비우 - 5 - 량등급의 회사채 가산금...,NaN,NaN,NaN,6,4,159,0,0,0,"금융/NNG,불안사태/NNG,발생/NNG,하/VV,가운데/NNG,국제/NNG,환투기..."
6,2005-10-11,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"일부 위원은 지방자치단체의 투자유치활동 이 활발해지고 항공기, 철도, 자동차 등의...",일부 위원은 미국과 우리나라의 정책금 리 격차가 확대되고 있는 가운데 금년 두 차...,일부 위원은 8월말 이후 우리나라 주가지수가 큰 폭 상승한 것은 경제성장에 대한...,NaN,NaN,NaN,11,4,158,0,0,0,"금리갭/NNG,확대/NNG,되/XSV,가운데/NNG,남/VV,fed/NNG,금리/N..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,2017-10-19,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 이번 경제전망을 보면 그동안 경기회복에도 불구하고 낮아졌던 근원물가상...,일부 위원은 최근 미국에서는 임금상승세가 확대되고 세제개혁에 대한 기대 감이 커지...,NaN,일부 위원은 유휴생산능력과 물가 간의 관계를 분석함에 있어서는 GDP갭뿐 만 아니...,일부 위원은 이번 회의에서는 기준금리를 현 1.25%에서 1.5%로 조정하는 것 ...,NaN,54,44,0,20,156,0,"임금/NNG,상승/NNG,확대/NNG,되/XSV,세제/NNG,개혁/NNG,기대/NN..."
253,2017-01-13,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 대내외 여건이 앞으로의 성장경로를 논하기가 어려울 만큼 매우 불확실한...,일부 위원은 미국의 시간당평균임금 상승률이 확대되고 있어 물가상승 압력 이 높아질...,"일부 위원은 관련부서에서는 금년중 가계대출이 은행, 비은행 모두 증가세가 둔화되겠...",일부 위원은 부동산경기나 금융사이클이 실물경기와 괴리되어 있는 것은 아 닌지 심도...,일부 위원은 다음 통화정책방향 결정 시까지 기준금리를 현 1.25%에서 유지 하여...,NaN,55,14,15,16,157,0,"임금/NNG,상승률/NNG,확대/NNG,되/XSV,물가/NNG,상승/NNG,압력/N..."
255,2017-11-30,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 9∼10월 실물경제 지표를 묶어서 보면 서비스업 생산이 늘어나 는 가...,일부 위원은 최근 미국의 금융·경제상황을 보면 금년 12월 미 연준이 금리를 인상...,NaN,일부 위원은 수출 증가세가 높더라도 가계부채 등 구조적 요인으로 인해 민 간소비로...,일부 위원은 지난 금통위에서 언급하였던 금리인상의 요인들이 경제흐름에 의해 확인되...,NaN,61,34,0,18,147,0,"금융경제/NNG,보/VV,fed/NNG,금리/NNG,인상/NNG,하/VV,예견/NN..."
258,2017-02-23,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 금년 1월 소비자물가가 전년동월대비 2.0% 상승하는 등 예상보 다 ...,일부 위원은 Yellen 미 연준 의장이 미국의 금리인상 지연은 바람직하지 않다 ...,NaN,일부 위원은 일반인 기대인플레이션이 물가목표보다 높은 수준을 지속해 오 다가 최근...,일부 위원은 다음 통화정책방향 결정시까지 기준금리를 현 수준 1.25%로 유 지하...,NaN,46,51,0,17,157,0,"금리/NNG,인상/NNG,지연/NNG,바람직/NNG,않/VX,언급/NNG,하/VV,..."


### 콜금리 라벨 붙이기

In [49]:
labeled_minutes_csv = pd.merge(call_rate_csv, new_minutes_df, on = "date")
labeled_minutes_csv

,date,call_rate,call_label,minutes,Economic Situation,Foreign Currency,Financial Markets,Monetary Policy,Participant Views,Government View,Economic Situation count,Foreign Currency count,Financial Markets count,Monetary Policy count,Participant Views count,Government View count,ngram
0,2005-06-09,3.28,decrease,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 소비재판매액 증가세가 다소 둔화되고 있으나 GDP 상의 소비와...,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,"일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 접한 연관성을 지니는지,...",NaN,NaN,NaN,19,4,160,0,0,0,"경제/NNG,경제/NNG,비해/VV,더/MAG,낫/VV,특별/NNG,요인/NNG,없..."
1,2005-07-07,3.03,decrease,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 이란의 새 대통령 취임.@@@이후 자국경제 보호주의 강화 등으로 ...,일부 위원은 6월 들어 국제유가가 크게 오르고 원화와 엔화간 동조화 현상이 뚜렷...,일부 위원은 단기적으로 대출이 급격히 증가 한 후에는 반드시 조정이 이어지고 민간...,NaN,NaN,NaN,13,5,160,0,0,0,"들/VV,국제/NNG,유가/NNG,크/VA,오르/VV,원화/NNG,엔/NNG,화간/..."
2,2005-08-11,3.26,increase,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"일부 위원은 소비, 수출, 건설투자가 견조한 증가세를 보이고 물가도 안정세를 이...",일부 위원은 위안화 절상 이후 아시아 통화중 우리나라 원화의 절상폭이 가장 컸다...,일부 위원은 최근 장기시장금리 상승에도 불 구하고 기업 자금조달은 오히려 확대되고...,NaN,NaN,NaN,11,12,159,0,0,0,"위안/NNG,절상/NNG,원화/NNG,절상/NNG,가장/MAG,크/VA,추가/NNG..."
3,2005-09-08,3.24,decrease,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 7월 전망시보다 높은 수준을 지속하고 있는 국제유가가 내년도 경제성...,일부 위원은 최근 인도네시아 금융불안 사태가 발생한 가운데 국제 환투기세력이 인...,일부 위원은 시장이 제대로 작동한다면 비우 - 5 - 량등급의 회사채 가산금...,NaN,NaN,NaN,6,4,159,0,0,0,"금융/NNG,불안사태/NNG,발생/NNG,하/VV,가운데/NNG,국제/NNG,환투기..."
4,2005-10-11,3.50,increase,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"일부 위원은 지방자치단체의 투자유치활동 이 활발해지고 항공기, 철도, 자동차 등의...",일부 위원은 미국과 우리나라의 정책금 리 격차가 확대되고 있는 가운데 금년 두 차...,일부 위원은 8월말 이후 우리나라 주가지수가 큰 폭 상승한 것은 경제성장에 대한...,NaN,NaN,NaN,11,4,158,0,0,0,"금리갭/NNG,확대/NNG,되/XSV,가운데/NNG,남/VV,fed/NNG,금리/N..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2017-05-25,1.22,decrease,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 금년 1/4분기 수출이 높은 증가율을 기록하였는데 단가 상승뿐만 아니...,일부 위원은 최근 국제금융시장이 안정된 모습을 지속하는 가운데 국내금융 시장도 외...,NaN,"일부 위원은 낮은 수준의 중립금리 지속은 금융안정 리스크 확대, 자원배분 왜곡 등...",일부 위원은 다음 통화정책방향 결정시까지 기준금리를 현 수준 1.25%로 유 지하...,NaN,55,31,0,22,157,0,"국제/NNG,금융시장/NNG,안정/NNG,되/VV,지속/NNG,하/XSV,가운데/N..."
119,2017-07-13,1.24,increase,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"일부 위원은 이번 성장전망과 관련하여 금년 전망치 달성은 무난해 보이지만, 내년에...",일부 위원은 주요국에서는 성장과 고용이 회복국면의 정점을 향해 가고 있음.@@@에도...,NaN,일부 위원은 향후 미 연준의 보유자산 축소가 정책금리 인상과 병행하여 시 행될 것...,일부 위원은 이번 회의에서는 기준금리를 지금의 1.25%로 동결하는 것이 바 람직...,NaN,44,27,0,7,141,0,"성장/NNG,고용/NNG,회복국면/NNG,정점/NNG,향하/VV@@@불구/NNG,물..."
120,2017-08-31,1.27,increase,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"일부 위원은 관련부서에서는 세계경기의 회복세 강화, 추경 집행 등 상방 리 스크 ...",일부 위원은 미국의 금리 정상화 행보에도 불구하고 미 달러화가 약세기조를 지속하면...,NaN,일부 위원은 비록 경기가 회복되고는 있지만 GDP갭과 물가갭은 여전히 마이 너스 ...,일부 위원은 이번에는 기준금리를 현 수준에서 유지하면서 경기 및 물가 관 련 불확...,NaN,42,38,0,8,149,0,"금리정상화/NNG,행보/NNG,불구/NNG,미/NNG,달러/NNG,약세/NNG,기조..."
121,2017-10-19,1.24,0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 이번 경제전망을 보면 그동안 경기회복에도 불구하고 낮아졌던 근원물가상...,일부 위원은 최근 미국에서는 임금상승세가 확대되고 세제개혁에 대한 기대 감이 커지...,NaN,일부 위원은 유휴생산능력과 물가 간의 관계를 분석함에 있어서는 GDP갭뿐 만 아니...,일부 위원은 이번 회의에서는 기준금리를 현 1.25%에서 1.5%로 조정하는 것 ...,NaN,54,44,0,20,156,0,"임금/NNG,상승/NNG,확대/NNG,되/XSV,세제/NNG,개혁/NNG,기대/NN..."


In [51]:
labeled_minutes_csv.to_csv('labeled_bok_minutes.csv')

In [50]:
pwd

'/content/drive/My Drive/bok'